<a href="https://colab.research.google.com/github/xxp-nlp/add_tag_algorithm/blob/main/%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8

In [ ]:
# CRF++のソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
FILE_NAME = "crfpp.tar.gz"
!wget 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O $FILE_NAME
!tar xvf crfpp.tar.gz
%cd CRF++-0.58
!./configure && make && make install && ldconfig
%cd ..

In [ ]:
# CaboChaのソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
FILE_NAME = "cabocha-0.69.tar.bz2"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt
!tar -xvf cabocha-0.69.tar.bz2
%cd cabocha-0.69
!./configure -with-charset=utf-8 && make && make check && make install && ldconfig
%cd ..

In [18]:
# 読みやすい文　係り受け解析
!cabocha -f1 -o data_correct.txt.parsed data_correct.txt

In [30]:
import re

# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):

        #タブとカンマで分割
        cols = separator.split(line)

        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.srcs = []   # 係り元文節インデックス番号のリスト
        self.dst  = dst  # 係り先文節インデックス番号
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.strip('、')

# 係り元を代入し、Chunkリストを文のリストを追加
def append_sentence(chunks, sentences):

    # 係り元を代入
    for i, chunk in enumerate(chunks):
        if chunk.dst != -1:
            chunks[chunk.dst].srcs.append(i)
    sentences.append(chunks)
    return sentences, []

morphs = []
chunks = []
sentences = []

with open('/content/data_correct.txt.parsed') as f:

    for line in f:
        dependancies = dependancy.match(line)

        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))

        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []

        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))

        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            sentences, chunks = append_sentence(chunks, sentences)

for i, chunk in enumerate(sentences[0]):
    print('{}: {}, 係り先:{}, 係り元:{}'.format(i, chunk.trim_phrase, chunk.dst, chunk.srcs))

0: 同社では, 係り先:5, 係り元:[]
1: すでに, 係り先:5, 係り元:[]
2: 準役員クラス以上に, 係り先:5, 係り元:[]
3: 能力重視型の, 係り先:4, 係り元:[]
4: 年俸制を, 係り先:5, 係り元:[3]
5: 導入している。, 係り先:-1, 係り元:[0, 1, 2, 4]


In [29]:
# 読みにくい文　係り受け解析
!cabocha -f1 -o data_wrong.txt.parsed data_wrong.txt

In [20]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)

class Morph:
    def __init__(self, line):

        #タブとカンマで分割
        cols = separator.split(line)

        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.srcs = []   # 係り元文節インデックス番号のリスト
        self.dst  = dst  # 係り先文節インデックス番号
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.strip('、')        
        self.tag = 'keep'

# 係り元を代入し、Chunkリストを文のリストを追加
def append_sentence(chunks, sentences):

    # 係り元を代入
    for i, chunk in enumerate(chunks):
        if chunk.dst != -1:
            chunks[chunk.dst].srcs.append(i)
    sentences.append(chunks)
    return sentences, []

morphs = []
chunks = []
sentences_w = []

with open('/content/data_wrong.txt.parsed') as f:

    for line in f:
        dependancies = dependancy.match(line)

        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))

        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []

        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))

        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            sentences_w, chunks = append_sentence(chunks, sentences_w)

for i, chunk in enumerate(sentences_w[335]):
    print('{}: {}: {}'.format(i, chunk.trim_phrase, chunk.tag))

0: また: keep
1: チェチェン側の: keep
2: 捕虜に: keep
3: 七日: keep
4: 首都の: keep
5: 南西: keep
6: 三十キロの: keep
7: 村で: keep
8: 精鋭である: keep
9: ロシア軍の: keep
10: 空てい部隊が: keep
11: なったとも: keep
12: 伝えられており: keep
13: 依然: keep
14: ロシア軍に対する: keep
15: チェチェン側の: keep
16: 抵抗は: keep
17: 各地で: keep
18: 続いている: keep
19: 模様だ。: keep


In [21]:
dic2 = {}
for i in range(10):
  for j, chunk in enumerate(sentences_w[i]):
    if sentences_w[i][j].trim_phrase not in dic2:
      dic2[sentences_w[i][j].trim_phrase] = []
      
    dic2[sentences_w[i][j].trim_phrase].append(j)
    print(sentences_w[i][j].trim_phrase,dic2[sentences_w[i][j].trim_phrase])
for i in range(10):
  for j, chunk in enumerate(sentences[i]):
    if j == len(sentences[i])-1:
      break

    for m in range(len(dic2[sentences[i][j].trim_phrase])):
      for n in range(len(dic2[sentences[i][j+1].trim_phrase])):

        if sentences[i][j].dst == j+1 and dic2[sentences[i][j].trim_phrase][m] == dic2[sentences[i][j+1].trim_phrase][n] - 1: 
          sentences_w[i][dic2[sentences[i][j].trim_phrase][m]].tag = "reduce"

        if dic2[sentences[i][j].trim_phrase][m] < len(dic2)-1 and dic2[sentences[i][j].trim_phrase][m] == dic2[sentences[i][j+1].trim_phrase][n] + 1: 
          sentences_w[i][dic2[sentences[i][j].trim_phrase][m]].tag = "swap_B"
          sentences_w[i][dic2[sentences[i][j+1].trim_phrase][n]].tag = "swap_F"

    print(j,sentences[i][j].trim_phrase,sentences[i][j].dst)
  print("------------------------")

同社では [0]
準役員クラス以上に [1]
能力重視型の [2]
年俸制を [3]
すでに [4]
導入している。 [5]
党全体での [0]
新党移行を [1]
今春の [2]
統一地方選後に [3]
目指す [4]
考えを [5]
強調 [6]
離党など [7]
性急な [8]
行動への [9]
自重を [10]
首相は [11]
求めた。 [12]
しかし [0]
二十日の [1]
通常国会召集前の [2]
新党結成を [3]
山花氏は [4]
主張して [5]
物別れに [6]
終わり [7]
同党の [8]
亀裂は [9]
決定的になった。 [10]
山花氏は [4, 0]
準備会参加議員の [1]
一回目の [2]
集約を [3]
九日 [4]
予定通り [5]
行う。 [6]
しかし [0, 0]
準備会参加は [1]
離党を [2]
ただちに [3]
意味せず [4]
焦点に [5]
実際の [6]
離党者数が [7]
なる。 [8]
妥協案として [0]
市民団体代表も [1]
交えた [2]
新党準備会を [3]
統一地方選後に [3, 4]
発足させる [5]
考えを [5, 6]
示したが [7]
歩み寄りは [8]
なかった。 [9]
また [0]
村山政権を [1]
通常国会で [2]
新党による [3]
新国会内会派を [4]
旗揚げしても [5]
支える [6]
姿勢を [7]
山花氏は [4, 0, 8]
表明。 [9]
しかし [0, 0, 0]
倒閣に [1]
山花氏らの [2]
行動は [3]
つながるとの [4]
考えを [5, 6, 5]
首相は [11, 6]
強調した。 [7]
さらに [0]
首相は [11, 6, 1]
政権基盤を [2]
揺るがすような [3]
行動を [4]
訪米中に [5]
取らない [6]
よう [7]
くぎを [8]
刺した。 [9]
公式に [0]
ロシア政府が [1]
認めたのは [2]
同大統領の [3]
首都脱出は [4]
先に [5]
六日 [6]
グロズヌイの [7]
軍事筋の [8]
情報として [9]
インタファクス通信が [10]
伝えたが [11]
初めて。 [12]
0 同社では 5
1 すでに 5
2 準役員クラス以上に 5
3 能力重視型の 4
4 年俸制を 5

In [28]:
for i in range(10):
  for j in range(len(sentences_w[i])):
    print(sentences_w[i][j].phrase,sentences_w[i][j].tag)
  print("------------------------")

同社では、 keep
準役員クラス以上に keep
能力重視型の reduce
年俸制を keep
すでに keep
導入している。 keep
------------------------
党全体での reduce
新党移行を keep
今春の reduce
統一地方選後に keep
目指す reduce
考えを reduce
強調、 keep
離党など keep
性急な reduce
行動への reduce
自重を keep
首相は keep
求めた。 keep
------------------------
しかし、 keep
二十日の reduce
通常国会召集前の reduce
新党結成を keep
山花氏は keep
主張して keep
物別れに reduce
終わり、 keep
同党の reduce
亀裂は reduce
決定的になった。 keep
------------------------
山花氏は、 keep
準備会参加議員の reduce
一回目の reduce
集約を keep
九日、 swap_F
予定通り swap_B
行う。 keep
------------------------
しかし、 keep
準備会参加は keep
離党を swap_F
ただちに swap_B
意味せず、 keep
焦点に keep
実際の reduce
離党者数が keep
なる。 keep
------------------------
妥協案として、 keep
市民団体代表も reduce
交えた reduce
新党準備会を keep
統一地方選後に keep
発足させる reduce
考えを reduce
示したが、 keep
歩み寄りは reduce
なかった。 keep
------------------------
また、 keep
村山政権を、 keep
通常国会で keep
新党による reduce
新国会内会派を reduce
旗揚げしても keep
支える reduce
姿勢を、 keep
山花氏は keep
表明。 keep
------------------------
しかし、 keep
倒閣に keep
山花氏らの reduce
行動は keep
つながるとの reduce
考えを keep
首相は reduce
